In [ ]:
from webScraper import *

<font size="5">
    Step 1: Scrape Latest Articles
</font>

In [ ]:
# Scrape latest articles from https://nos.nl
# Outputs latest articles in 'articles_scrapped.json'
webDataScrape()

<font size="5">
    Step 2: Add to Master Article List
</font>

In [ ]:
# Updates master_article_list.json, using articles_scrapped.json as input
updateMasterList()

In [ ]:
# Check length of master article list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))
print(len(master_article_list))

<font size="5">
    Step 3: Convert text in Articles to list of words with frequency
</font>

In [ ]:
# Retrieve all articles from master list
master_article_list = []
with open('master_article_list.json') as json_file:
    master_article_data = json.load(json_file)
    for article in master_article_data:
        master_article_list.append(Article(article['ID'], article['url'], article['title'], article['date_time'], article['text']))

In [ ]:
# Retrieve list of words from each article
full_word_list = []
for article in master_article_list:
    full_word_list.append(article.retrieveWordList())
print(full_word_list)